In [76]:
!pip install elasticsearch

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch

C:\Users\Honor\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Honor\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Honor\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
es = Elasticsearch('http://localhost:9200/')

In [4]:
es.ping()

True

In [23]:
import tensorflow

In [26]:
pip show Pillow


Name: Pillow
Version: 10.0.1
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Jeffrey A. Clark (Alex)
Author-email: aclark@aclark.net
License: HPND
Location: c:\users\honor\anaconda3\lib\site-packages
Requires: 
Required-by: bokeh, datashader, imageio, matplotlib, pdfplumber, pytesseract, roboflow, scikit-image, supervision, torchvision, ultralytics
Note: you may need to restart the kernel to use updated packages.


In [81]:
pip show elasticsearch

Name: elasticsearch
Version: 8.11.0
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: Elastic Client Library Maintainers
Author-email: client-libs@elastic.co
License: Apache-2.0
Location: c:\users\honor\anaconda3\lib\site-packages
Requires: elastic-transport
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Prepare the data

In [5]:
df = pd.read_csv("data/medical_tc_train.csv").loc[:100]

In [6]:
# df_pm = pd.read_csv("data/Pubmed_200k_RCT/train.csv").loc[:100]

In [7]:
df.head()

,condition_label,medical_abstract
0,5,Tissue changes around loose prostheses. A cani...
1,1,Neuropeptide Y and neuron-specific enolase lev...
2,2,"Sexually transmitted diseases of the colon, re..."
3,1,Lipolytic factors associated with murine and h...
4,3,Does carotid restenosis predict an increased r...


In [8]:
df.isna().sum().sum()

0

# Convert text to Vector using S-BERT Model

In [9]:
from tqdm import tqdm
tqdm.pandas()


In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
df["description_vector"] = df["medical_abstract"].progress_apply(lambda x: model.encode(x))

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [02:38<00:00,  1.57s/it]


In [13]:
df.head()

,condition_label,medical_abstract,description_vector
0,5,Tissue changes around loose prostheses. A cani...,"[-0.0076953364, -0.029528731, 0.009244567, -0...."
1,1,Neuropeptide Y and neuron-specific enolase lev...,"[0.049338676, -0.00904814, 0.06583269, 0.01739..."
2,2,"Sexually transmitted diseases of the colon, re...","[0.055318918, 0.028947582, -0.00078177685, -0...."
3,1,Lipolytic factors associated with murine and h...,"[0.051143337, 0.0036285874, 0.053873442, -0.00..."
4,3,Does carotid restenosis predict an increased r...,"[0.010875214, 0.030527258, 0.0020399122, -0.02..."


In [14]:
es.ping()

True

# Create new index

In [15]:
from indexMapping import indexMapping

In [16]:
es.indices.create(index="all_documents", mappings= indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_documents'})

In [17]:
records_list = df.to_dict("records")

In [18]:
records_list[0]["condition_label"]

5

In [19]:
for record in records_list:
    try:
        es.index(index="all_documents", document=record)
    except Exception as e:
        print(e)

In [20]:
es.count(index="all_documents")

ObjectApiResponse({'count': 101, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

# Searching documents:

In [21]:
user_input = "football"
vector_input = model.encode(user_input)

query = {
    "field": "description_vector",
    "query_vector": vector_input,
    "k":5,
    "num_candidates": 100,
}

res = es.knn_search(index="all_documents", knn=query, source=["medical_abstract"])

res["hits"]["hits"]

C:\Users\Honor\AppData\Local\Temp\ipykernel_20212\1266375494.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_documents", knn=query, source=["medical_abstract"])


[{'_index': 'all_documents',
  '_id': 'nPkXTYwBIH9pooElEBJp',
  '_score': 0.42266774,
  '_source': {'medical_abstract': 'The prolonged burner syndrome. Over the course of a single football season, six players evaluated by the medical staff had burners that displayed a prolonged neurologic recovery. These players were examined and subsequently evaluated with isokinetic testing and electrodiagnostic studies to elucidate better the short-term natural history of the prolonged burner syndrome. Evidence of muscular weakness at 72 hours postinjury best correlated with positive electrodiagnostic findings. No correlation was found between the initial physical examination findings and the results of electrodiagnostic testing. Isokinetic strength evaluation demonstrated many relative strength differences that were difficult to discern with manual muscle testing. The return of a player to athletic competition following this injury should largely be based on the clinical examination. '}},
 {'_index